## Observations

<b>Positive</b>
- Non Null values for all columns except Primary Topic


<b>Negative</b>
- Data types are not correct.

  Session_id should be object

  Visit date should be date_time

- "Traffic Source" has same value but different spellings
- Primary topic has NULL values
- Need to understand what "Page title" and "Actions" are.
- "Primary Topic" has different spellings for same word. There are special characters like (; / 4345 test)
- "Page Title" has data issues. Same word has different spellings.

In [1]:
import pandas as pd

In [2]:
web_path = "../data/Web_Traffic.csv"
web_df = pd.read_csv(web_path, parse_dates=["Visit Date"])
web_df.head()

/tmp/ipykernel_97775/1231117421.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  web_df = pd.read_csv(web_path, parse_dates=["Visit Date"])


,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,2023-01-01,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1,11002963,5,2023-01-01,Organic,Srcoll 50%,13,Contact,00v2000000KLfqsAAD,Fund Universe
2,11000521,7,2023-01-01,Google,Srcoll 50%,3,Home,00v2000000KO68GAAT,Fund Universe
3,11001502,5,2023-01-01,LinkedIn,Download,14,Contact,00v2000000KrZiaAAF,Fund Universe
4,11001448,6,2023-01-01,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe


In [3]:
web_df.dtypes

Session ID                          int64
Pageviews                           int64
Visit Date                 datetime64[ns]
Traffic Source                     object
Actions                            object
Time spent (in minutes)             int64
Page Title                         object
Contact ID                         object
Primary Topic                      object
dtype: object

In [4]:
web_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428332 entries, 0 to 428331
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Session ID               428332 non-null  int64         
 1   Pageviews                428332 non-null  int64         
 2   Visit Date               428332 non-null  datetime64[ns]
 3   Traffic Source           428174 non-null  object        
 4   Actions                  428332 non-null  object        
 5   Time spent (in minutes)  428332 non-null  int64         
 6   Page Title               428332 non-null  object        
 7   Contact ID               428332 non-null  object        
 8   Primary Topic            291885 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 29.4+ MB


In [5]:
web_df.describe(include = "all")

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
count,4.283320e+05,428332.000000,428332,428174,428332,428332.000000,428332,428332,291885
unique,NaN,NaN,NaN,6,8,NaN,7,3872,10
top,NaN,NaN,NaN,LinkedIn,Srcoll 50%,NaN,Home,00v2000000KXVZoAAP,Funds
freq,NaN,NaN,NaN,179156,80518,NaN,102632,140,85450
mean,1.100176e+07,4.000712,2023-07-25 09:03:26.856364288,NaN,NaN,8.006791,NaN,NaN,NaN
min,1.100011e+07,1.000000,2023-01-01 00:00:00,NaN,NaN,1.000000,NaN,NaN,NaN
25%,1.100094e+07,2.000000,2023-04-13 00:00:00,NaN,NaN,4.000000,NaN,NaN,NaN
50%,1.100176e+07,4.000000,2023-07-25 00:00:00,NaN,NaN,8.000000,NaN,NaN,NaN
75%,1.100258e+07,6.000000,2023-11-06 00:00:00,NaN,NaN,12.000000,NaN,NaN,NaN
max,1.100340e+07,7.000000,2024-02-16 00:00:00,NaN,NaN,15.000000,NaN,NaN,NaN


### Duplicate Analysis

- No duplicates

In [6]:
web_df[web_df.duplicated()]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic


### Column value counts

In [7]:
web_df["Traffic Source"].value_counts()

Traffic Source
LinkedIn    179156
Organic      88277
Youtube      62203
Google       60922
YouTube      30530
organic       7086
Name: count, dtype: int64

In [8]:
web_df["Actions"].value_counts()

Actions
Srcoll 50%     80518
Download       80195
Scroll 25%     53731
Video          53545
Login          53507
CTA            53450
Scroll 100%    26820
Scroll 75%     26566
Name: count, dtype: int64

In [9]:
web_df["Page Title"].value_counts()

Page Title
Home          102632
Funds          85450
Navigation     85356
Strategy       51686
Contact        34595
Invest         34309
funds          34304
Name: count, dtype: int64

In [10]:
web_df[["Page Title", "Actions"]].value_counts().sort_index()

Page Title  Actions    
Contact     CTA             4313
            Download        6505
            Login           4338
            Scroll 100%     2131
            Scroll 25%      4340
            Scroll 75%      2137
            Srcoll 50%      6546
            Video           4285
Funds       CTA            10616
            Download       16149
            Login          10659
            Scroll 100%     5422
            Scroll 25%     10675
            Scroll 75%      5332
            Srcoll 50%     15972
            Video          10625
Home        CTA            12790
            Download       19083
            Login          12804
            Scroll 100%     6388
            Scroll 25%     13012
            Scroll 75%      6364
            Srcoll 50%     19328
            Video          12863
Invest      CTA             4219
            Download        6420
            Login           4312
            Scroll 100%     2173
            Scroll 25%      4303
            Scroll 

In [11]:
web_df["Primary Topic"].value_counts()

Primary Topic
Funds            85450
Navigation       85356
Strategy         51686
Invest           34309
funds            34304
Fund Universe      650
/                   96
;                   32
4345                 1
test                 1
Name: count, dtype: int64

### Primary Topic   / ; 4345 test

- Page Title is mapped to Primary Topic
- All except Home and Contact works fine. They are majorly NULL. Some of them are filled with Fund Universe, /, ; , 4345, test
- test entry can be removed
- Assign "Undefined" topic to Home and Contact Page title

In [12]:
web_df[web_df["Primary Topic"].notnull()][["Page Title", "Primary Topic"]].value_counts()

Page Title  Primary Topic
Funds       Funds            85450
Navigation  Navigation       85356
Strategy    Strategy         51686
Invest      Invest           34309
funds       funds            34304
Home        Fund Universe      490
Contact     Fund Universe      160
Home        /                   74
            ;                   23
Contact     /                   22
            ;                    9
            4345                 1
Home        test                 1
Name: count, dtype: int64

In [13]:
web_df[web_df["Primary Topic"].isnull()]["Page Title"].value_counts()

Page Title
Home       102044
Contact     34403
Name: count, dtype: int64

### Session ID 

In [14]:
web_df.head()

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,2023-01-01,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1,11002963,5,2023-01-01,Organic,Srcoll 50%,13,Contact,00v2000000KLfqsAAD,Fund Universe
2,11000521,7,2023-01-01,Google,Srcoll 50%,3,Home,00v2000000KO68GAAT,Fund Universe
3,11001502,5,2023-01-01,LinkedIn,Download,14,Contact,00v2000000KrZiaAAF,Fund Universe
4,11001448,6,2023-01-01,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe


In [15]:
web_df[web_df["Session ID"] == 11002649]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,2023-01-01,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1217,11002649,5,2023-01-02,LinkedIn,Scroll 100%,7,funds,00v2000000KWVjeAAH,funds
4088,11002649,1,2023-01-04,Organic,Video,8,funds,00v2000000KLpkHAAT,funds
6631,11002649,4,2023-01-07,LinkedIn,Scroll 75%,4,Funds,00v2000000KMNBTAA5,Funds
11504,11002649,3,2023-01-11,LinkedIn,Srcoll 50%,11,funds,00v2000000ZCEVVAA5,funds
...,...,...,...,...,...,...,...,...,...
426263,11002649,4,2024-02-15,Google,CTA,11,funds,00v2000000KNzJUAA1,funds
426554,11002649,7,2024-02-15,LinkedIn,CTA,6,Navigation,00v2000000TKvPaAAL,Navigation
427901,11002649,6,2024-02-16,Organic,Download,1,Funds,00v2000000SJKgSAAX,Funds
427944,11002649,3,2024-02-16,Organic,CTA,6,Funds,00v2000000OSlV4AAL,Funds


In [16]:
web_df[web_df["Contact ID"] == "00v2000000KM7EdAAL"]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,2023-01-01,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
6313,11001732,5,2023-01-06,Google,Scroll 75%,4,Invest,00v2000000KM7EdAAL,Invest
7024,11000241,2,2023-01-07,Youtube,Download,1,Home,00v2000000KM7EdAAL,NaN
10416,11003116,5,2023-01-11,LinkedIn,Download,2,Funds,00v2000000KM7EdAAL,Funds
14035,11000165,2,2023-01-14,LinkedIn,Scroll 75%,9,Home,00v2000000KM7EdAAL,NaN
...,...,...,...,...,...,...,...,...,...
418679,11002626,5,2024-02-07,YouTube,Download,3,Funds,00v2000000KM7EdAAL,Funds
419187,11000810,6,2024-02-08,LinkedIn,CTA,8,Navigation,00v2000000KM7EdAAL,Navigation
422591,11001145,4,2024-02-11,LinkedIn,Video,8,Navigation,00v2000000KM7EdAAL,Navigation
424821,11003049,7,2024-02-13,Google,Download,13,Home,00v2000000KM7EdAAL,NaN


In [17]:
web_df[web_df["Session ID"] == 11002649]["Contact ID"].value_counts()

Contact ID
00v2000000KZAdeAAH    2
00v2000000KXc97AAD    2
00v2000000gTNqYAAW    2
00v2000000KMF2vAAH    2
00v2000000KrhxSAAR    2
                     ..
00v2000000KXimcAAD    1
00v2000000MibWDAAZ    1
00v2000000M5ajtAAB    1
00v2000000KXvfnAAD    1
00v2000000dyEtdAAE    1
Name: count, Length: 126, dtype: int64

In [18]:
contact_path = "../data/Contact_details.csv"
contact_df = pd.read_csv(contact_path)

In [19]:
unique_contacts = web_df[web_df["Session ID"] == 11002649]["Contact ID"].unique()

contact_df[contact_df["Contact_id"].isin(unique_contacts)]

,Contact Name,Company_name,Contact_id,Local Salutation,Region,Country
18,Abra Boris,Bloomberg,00v2000000Mhvw9AAB,Dear Abra Boris,EMEA,UK
46,Alvarado Bowman,Deloitte & Touche,00v2000000bFleNAAS,Dear Alvarado Bowman,EMEA,United Kingdom
57,Alec Contreras,Diebold Inc.,00v2000000cVOx3AAG,Dear Alec Contreras,EMEA,United Kingdom
112,Amelia Whitley,Kemper Insurance,00v2000000IZ0FkAAL,Dear Amelia Whitley,EMEA,Uk
120,acqueline Barnett,Liberty Mutual Insurance,00v2000000MOJDNAA5,Dear acqueline Barnett,EMEA,Uk
...,...,...,...,...,...,...
3784,Raymond Rollins,HSBC,00v2000000KsAcFAAV,Dear Raymond Rollins,AMER,Canada
3787,Patel Wilder,HSBC,00v2000000KsAjEAAV,Dear Patel Wilder,AMER,Canada
3792,Craig Bates,HSBC,00v2000000KrhxSAAR,Dear Craig Bates,AMER,Canada
3813,Barrera Stokes,HSBC,00v2000000Ka4lrAAB,Dear Barrera Stokes,AMER,Canada


### Pageviews

In [20]:
web_df[(web_df["Session ID"] == 11002649) \
& (web_df["Contact ID"] == "00v2000000KZAdeAAH")]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
276351,11002649,3,2023-09-23,LinkedIn,Scroll 25%,11,Home,00v2000000KZAdeAAH,NaN
327494,11002649,7,2023-11-12,Youtube,Scroll 25%,7,Home,00v2000000KZAdeAAH,NaN


In [21]:
web_df[web_df["Contact ID"] == "00v2000000KZAdeAAH"].sort_values(by="Visit Date")

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
238,11001969,6,2023-01-01,Google,Download,11,Home,00v2000000KZAdeAAH,Fund Universe
7656,11002882,6,2023-01-08,Youtube,Download,10,Invest,00v2000000KZAdeAAH,Invest
10375,11001215,3,2023-01-10,LinkedIn,Srcoll 50%,13,funds,00v2000000KZAdeAAH,funds
12065,11000994,2,2023-01-12,Organic,Video,6,Home,00v2000000KZAdeAAH,NaN
12280,11000937,3,2023-01-12,Google,Download,13,Funds,00v2000000KZAdeAAH,Funds
...,...,...,...,...,...,...,...,...,...
421462,11002887,5,2024-02-10,LinkedIn,Srcoll 50%,8,Strategy,00v2000000KZAdeAAH,Strategy
421834,11002046,4,2024-02-10,LinkedIn,Download,4,Navigation,00v2000000KZAdeAAH,Navigation
422016,11000374,6,2024-02-10,LinkedIn,Srcoll 50%,9,Navigation,00v2000000KZAdeAAH,Navigation
423603,11000775,3,2024-02-12,LinkedIn,Video,2,Home,00v2000000KZAdeAAH,NaN


In [22]:
web_df[(web_df["Contact ID"] == "00v2000000KZAdeAAH") \
& (web_df["Page Title"] == "Home")].sort_values(by=["Visit Date"])

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
238,11001969,6,2023-01-01,Google,Download,11,Home,00v2000000KZAdeAAH,Fund Universe
12065,11000994,2,2023-01-12,Organic,Video,6,Home,00v2000000KZAdeAAH,NaN
14779,11000966,7,2023-01-15,YouTube,Scroll 25%,5,Home,00v2000000KZAdeAAH,NaN
49470,11001693,3,2023-02-17,Youtube,Srcoll 50%,4,Home,00v2000000KZAdeAAH,NaN
63634,11001437,6,2023-03-03,LinkedIn,Scroll 25%,6,Home,00v2000000KZAdeAAH,NaN
80735,11000116,4,2023-03-19,YouTube,Video,10,Home,00v2000000KZAdeAAH,NaN
86607,11000627,5,2023-03-25,Google,Scroll 25%,11,Home,00v2000000KZAdeAAH,NaN
94427,11000192,4,2023-04-01,Youtube,Video,7,Home,00v2000000KZAdeAAH,NaN
95909,11001186,7,2023-04-02,Youtube,Srcoll 50%,9,Home,00v2000000KZAdeAAH,NaN
113697,11001793,3,2023-04-19,Google,Srcoll 50%,5,Home,00v2000000KZAdeAAH,NaN


In [23]:
web_df[["Session ID", "Contact ID"]].value_counts()

Session ID  Contact ID        
11001298    00v2000000MhkOZAAZ    4
11001315    00v2000000KXs3VAAT    3
11001986    00v2000000SJNPJAA5    3
11000889    00v2000000KLtx2AAD    3
11001598    00v2000000VkL8iAAF    3
                                 ..
11001218    00v2000000KLsCvAAL    1
            00v2000000KLpjiAAD    1
            00v2000000KLmcLAAT    1
            00v2000000KLjPAAA1    1
11003405    00vD000000h80bSIAQ    1
Name: count, Length: 421118, dtype: int64

In [24]:
web_df[(web_df["Session ID"] == 11001298) \
& (web_df["Contact ID"] == "00v2000000MhkOZAAZ")]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
84731,11001298,1,2023-03-23,LinkedIn,CTA,2,Strategy,00v2000000MhkOZAAZ,Strategy
164318,11001298,2,2023-06-07,LinkedIn,Video,6,funds,00v2000000MhkOZAAZ,funds
214767,11001298,7,2023-07-26,LinkedIn,Video,3,Funds,00v2000000MhkOZAAZ,Funds
318828,11001298,4,2023-11-03,LinkedIn,CTA,7,Navigation,00v2000000MhkOZAAZ,Navigation


In [25]:
web_df[(web_df["Contact ID"] == "00v2000000MhkOZAAZ")].sort_values(["Visit Date"])

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
185,11000510,1,2023-01-01,Youtube,CTA,7,Invest,00v2000000MhkOZAAZ,Invest
2480,11001724,5,2023-01-03,LinkedIn,Scroll 75%,14,Funds,00v2000000MhkOZAAZ,Funds
11910,11003346,3,2023-01-12,LinkedIn,Scroll 75%,15,Navigation,00v2000000MhkOZAAZ,Navigation
18047,11002608,4,2023-01-18,LinkedIn,CTA,3,Invest,00v2000000MhkOZAAZ,Invest
21367,11001984,4,2023-01-21,LinkedIn,Video,9,Invest,00v2000000MhkOZAAZ,Invest
...,...,...,...,...,...,...,...,...,...
416702,11003265,1,2024-02-05,YouTube,CTA,6,Strategy,00v2000000MhkOZAAZ,Strategy
418837,11001982,1,2024-02-07,Google,CTA,3,Navigation,00v2000000MhkOZAAZ,Navigation
420067,11001898,5,2024-02-09,Youtube,Download,8,Home,00v2000000MhkOZAAZ,NaN
424013,11003191,7,2024-02-12,LinkedIn,Scroll 75%,1,Contact,00v2000000MhkOZAAZ,NaN


In [26]:
web_df["unique_session_id"] = web_df["Session ID"].astype(str) + web_df["Contact ID"]

In [27]:
web_df[["unique_session_id", "Contact ID"]].value_counts()

unique_session_id           Contact ID        
1100129800v2000000MhkOZAAZ  00v2000000MhkOZAAZ    4
1100131500v2000000KXs3VAAT  00v2000000KXs3VAAT    3
1100198600v2000000SJNPJAA5  00v2000000SJNPJAA5    3
1100088900v2000000KLtx2AAD  00v2000000KLtx2AAD    3
1100159800v2000000VkL8iAAF  00v2000000VkL8iAAF    3
                                                 ..
1100121800v2000000KLsCvAAL  00v2000000KLsCvAAL    1
1100121800v2000000KLpjiAAD  00v2000000KLpjiAAD    1
1100121800v2000000KLmcLAAT  00v2000000KLmcLAAT    1
1100121800v2000000KLjPAAA1  00v2000000KLjPAAA1    1
1100340500vD000000h80bSIAQ  00vD000000h80bSIAQ    1
Name: count, Length: 421118, dtype: int64

In [28]:
web_df.groupby("unique_session_id")["Contact ID"].nunique()

unique_session_id
1100011200v2000000FP7CSAA1    1
1100011200v2000000FlKOOAA3    1
1100011200v2000000G5HEEAA3    1
1100011200v2000000GccIGAAZ    1
1100011200v2000000GcjnFAAR    1
                             ..
1100340500v2000000fHyn5AAC    1
1100340500v2000000gVCJrAAO    1
1100340500vD000000h1HJMIA2    1
1100340500vD000000h2mHrIAI    1
1100340500vD000000h80bSIAQ    1
Name: Contact ID, Length: 421118, dtype: int64

In [29]:
web_df[web_df["unique_session_id"] \
.isin(["1100129800v2000000MhkOZAAZ", "1100131500v2000000KXs3VAAT", "1100198600v2000000SJNPJAA5"])] \
.sort_values(["unique_session_id"])

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic,unique_session_id
84731,11001298,1,2023-03-23,LinkedIn,CTA,2,Strategy,00v2000000MhkOZAAZ,Strategy,1100129800v2000000MhkOZAAZ
164318,11001298,2,2023-06-07,LinkedIn,Video,6,funds,00v2000000MhkOZAAZ,funds,1100129800v2000000MhkOZAAZ
214767,11001298,7,2023-07-26,LinkedIn,Video,3,Funds,00v2000000MhkOZAAZ,Funds,1100129800v2000000MhkOZAAZ
318828,11001298,4,2023-11-03,LinkedIn,CTA,7,Navigation,00v2000000MhkOZAAZ,Navigation,1100129800v2000000MhkOZAAZ
56825,11001315,5,2023-02-24,Google,Srcoll 50%,1,Funds,00v2000000KXs3VAAT,Funds,1100131500v2000000KXs3VAAT
198801,11001315,7,2023-07-11,LinkedIn,Login,12,Funds,00v2000000KXs3VAAT,Funds,1100131500v2000000KXs3VAAT
389740,11001315,5,2024-01-11,LinkedIn,Login,2,Home,00v2000000KXs3VAAT,NaN,1100131500v2000000KXs3VAAT
11328,11001986,4,2023-01-11,Organic,Srcoll 50%,5,Strategy,00v2000000SJNPJAA5,Strategy,1100198600v2000000SJNPJAA5
88801,11001986,4,2023-03-27,LinkedIn,Download,15,Home,00v2000000SJNPJAA5,NaN,1100198600v2000000SJNPJAA5
407023,11001986,3,2024-01-27,Google,Login,1,Strategy,00v2000000SJNPJAA5,Strategy,1100198600v2000000SJNPJAA5


In [30]:
web_df[["unique_session_id", "Contact ID"]].value_counts()[web_df[["unique_session_id", "Contact ID"]].value_counts() > 1] 


unique_session_id           Contact ID        
1100129800v2000000MhkOZAAZ  00v2000000MhkOZAAZ    4
1100131500v2000000KXs3VAAT  00v2000000KXs3VAAT    3
1100198600v2000000SJNPJAA5  00v2000000SJNPJAA5    3
1100088900v2000000KLtx2AAD  00v2000000KLtx2AAD    3
1100159800v2000000VkL8iAAF  00v2000000VkL8iAAF    3
                                                 ..
1100257200v2000000SJUsSAAX  00v2000000SJUsSAAX    2
1100177300v2000000KMhcqAAD  00v2000000KMhcqAAD    2
1100216000v2000000bG0stAAC  00v2000000bG0stAAC    2
1100095600v2000000KuSMHAA3  00v2000000KuSMHAA3    2
1100094700v2000000fGnn3AAC  00v2000000fGnn3AAC    2
Name: count, Length: 7116, dtype: int64

In [31]:
web_df[web_df["Actions"] == "Login"]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic,unique_session_id
4,11001448,6,2023-01-01,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe,1100144800v2000000ez1ucAAA
12,11001440,1,2023-01-01,LinkedIn,Login,11,Navigation,00v2000000KLjPAAA1,Navigation,1100144000v2000000KLjPAAA1
15,11002936,7,2023-01-01,Youtube,Login,13,Funds,00v2000000Kra52AAB,Funds,1100293600v2000000Kra52AAB
17,11000602,4,2023-01-01,Youtube,Login,10,Funds,00v2000000XxS7iAAF,Funds,1100060200v2000000XxS7iAAF
23,11001302,7,2023-01-01,Organic,Login,8,Home,00v2000000KXiJdAAL,Fund Universe,1100130200v2000000KXiJdAAL
...,...,...,...,...,...,...,...,...,...,...
428289,11001560,1,2024-02-16,Youtube,Login,12,Navigation,00v2000000SlUM2AAN,Navigation,1100156000v2000000SlUM2AAN
428290,11002109,5,2024-02-16,LinkedIn,Login,2,funds,00v2000000I1O13AAF,funds,1100210900v2000000I1O13AAF
428297,11000315,5,2024-02-16,Youtube,Login,1,Funds,00v2000000KXht9AAD,Funds,1100031500v2000000KXht9AAD
428312,11001084,6,2024-02-16,Youtube,Login,6,Navigation,00v2000000I1O0xAAF,Navigation,1100108400v2000000I1O0xAAF


In [32]:
web_df["Session ID"].nunique()

3294

In [33]:
web_df["Contact ID"].nunique()

3872

In [34]:
web_df["unique_session_id"].nunique()

421118

In [35]:
web_df.groupby("Session ID")["Contact ID"].nunique()


Session ID
11000112    144
11000113    110
11000114    130
11000115    127
11000116    143
           ... 
11003401    142
11003402    107
11003403    136
11003404    140
11003405    137
Name: Contact ID, Length: 3294, dtype: int64

In [36]:
web_df.groupby("Session ID")["Contact ID"].value_counts() \
[web_df.groupby("Session ID")["Contact ID"].value_counts() > 1]


Session ID  Contact ID        
11000112    00v2000000IZ4yuAAD    2
            00v2000000KMhsEAAT    2
            00v2000000KXQmvAAH    2
            00v2000000ViqZNAAZ    2
11000113    00v2000000NdE1nAAF    2
                                 ..
11003404    00v2000000KZF3xAAH    2
            00v2000000WwZjLAAV    2
            00v2000000Yc6rzAAB    2
            00v2000000bGbnDAAS    2
11003405    00v2000000FEYKSAA5    2
Name: count, Length: 7116, dtype: int64

In [39]:
web_df["Visit Date"] = web_df["Visit Date"].astype(str)

In [40]:
web_df.head()

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic,unique_session_id
0,11002649,4,2023-01-01,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe,1100264900v2000000KM7EdAAL
1,11002963,5,2023-01-01,Organic,Srcoll 50%,13,Contact,00v2000000KLfqsAAD,Fund Universe,1100296300v2000000KLfqsAAD
2,11000521,7,2023-01-01,Google,Srcoll 50%,3,Home,00v2000000KO68GAAT,Fund Universe,1100052100v2000000KO68GAAT
3,11001502,5,2023-01-01,LinkedIn,Download,14,Contact,00v2000000KrZiaAAF,Fund Universe,1100150200v2000000KrZiaAAF
4,11001448,6,2023-01-01,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe,1100144800v2000000ez1ucAAA
